In [1]:
push!(LOAD_PATH,"./src/")
using gd2dist
using Random
using Distributions
using BenchmarkTools

┌ Info: Precompiling gd2dist [top-level]
└ @ Base loading.jl:1278


In [8]:
function mcmcSampleGammaStep!( data, datac, k, w, wc, θ, θc, κ, κc, steps = 1)
    
    #Declare auxiliar variables
    wAux = zeros(k[1])    
    wAuxc = zeros(k[1]*k[2])
    nAuxSum = zeros(k[1])
    nAuxSumc = zeros(k[2])
    nAux = zeros(k[1],k[2]+1)
    xAux = zeros(k[1],k[2]+1)
    xLogAux = zeros(k[1],k[2]+1)
    
    for i in 1:steps
        #Clean auxiliar
        nAux .= 0. 
        xAux .= 0.
        xLogAux .= 0.

        #SAMPLE IDENTITIES DATA
        for (j,d) in enumerate(data)
            #Weight normals
            for i in 1:k[1]
                wAux[i] = log(w[i])+loggamma(d,θ[i],κ[i])
            end
            wMax = maximum(wAux)
            wAux = exp.(wAux.-wMax)
            wAux /= sum(wAux)
            #Sample
            mult = Multinomial(1,wAux)
            chosen = findfirst(rand(mult).==1)
            #Add to auxiliar
            nAux[chosen,end] += 1
            nAuxSum[chosen] += 1
            xAux[chosen,end] += d
            xLogAux[chosen,end] += log(d)
        end

        #SAMPLE IDENTITIES DATA CONVOLVED
        for (j,d) in enumerate(datac)
            #Weight normals
            for i in 1:k[1]
                for ic in 1:k[2]
                    wAuxc[k[1]*(ic-1)+i] = log(w[i])+log(wc[ic])+loggamma(d,θ[i],κ[ic],θc[ic],κc[ic])
                end
            end
            wMax = maximum(wAuxc)
            wAuxc = exp.(wAuxc.-wMax)
            wAuxc /= sum(wAuxc)
            #Sample
            mult = Multinomial(1,wAuxc)
            chosenc = findfirst(rand(mult).==1)
            chosen = mod(chosenc-1,k[1])+1
            chosenc = Int((chosenc-chosen)/k[1])+1
            #Add to auxiliars
            nAuxSum[chosen] += 1
            nAuxSumc[chosenc] += 1
            nAux[chosen,chosenc] += 1
            xAux[chosen,chosenc] += d
            xLogAux[chosen,chosenc] += log(d)            
        end

        #SAMPLE WEIGHTS
        nAuxSum .+= 0.00000001
        dirich = Dirichlet(nAuxSum)
        w = rand(dirich)
        nAuxSumc .+= 0.00000001
        dirich = Dirichlet(nAuxSumc)
        wc = rand(dirich)

        #SAMPLE THETAS
        for i in 1:k[1]
            step = sum(nAux[i,:])
            if !(step ≈ 0.)
                θ[i]=sliceSamplingLog(loggammaθsum,θ[i],[xAux[i,:],xLogAux[i,:],nAux[i,:],κ[i],θc,κc])#,step=1/sqrt(step))
            end
        end
        for i in 1:k[2]
            step = sum(nAux[:,i])
            if !(step ≈ 0.)
                step = sum(nAux[:,i])
                θc[i]=sliceSamplingLog(loggammaθsum,θc[i],[xAux[:,i],nLogAux[:,i],nAux[i,:],κc[i],θ,κ])#,step=1/sqrt(step))
            end
        end

        #SAMPLE KAPPAS
        for i in 1:k[1]
            step = sum(nAux[i,:])
            if !(step ≈ 0.)
                κ[i]=sliceSamplingLog(loggammaκsum,κ[i],[xAux[i,:],xlogAux[i,:],nAux[i,:],θ[i],θc,κc])#,step=1/sqrt(step))
            end
        end
        for i in 1:k[2]
            step = sum(nAux[:,i])
            if !(step ≈ 0.)
                step = sum(nAux[:,i])
                κc[i]=sliceSamplingLog(loggammaκsum,κc[i],[xAux[:,i],xlogAux[:,i],nAux[:,i],θc[i],θ,κ])#,step=1/sqrt(step))
            end
        end
             
    end
        
    return
end

function gd2dist.mcmcSampleGamma( data, datac, k; chains::Int = 1, stopCriterion = :steps, steps::Int = 1000, ignore::Int=1000, gap::Int=1, basis = :Normal, initialCondition = Dict())

    #Declare parameters
    w = zeros(k[1])
    wc = zeros(k[2])
    θ = zeros(k[1])
    θc = zeros(k[2])
    κ = zeros(k[1])
    κc = zeros(k[2])
        
    #Declare storing dictionary
    out = Dict()
    out["w"] = zeros(steps*chains,k[1])
    out["θ"] = zeros(steps*chains,k[1])
    out["κ"] = zeros(steps*chains,k[1])
    out["wc"] = zeros(steps*chains,k[2])
    out["θc"] = zeros(steps*chains,k[2])
    out["κc"] = zeros(steps*chains,k[2])
    
    #Initialise model
    if initialCondition != Dict()
        w = initialCondition["w"]
        wc = initialCondition["wc"]
        θ = initialCondition["θ"]
        θc = initialCondition["θc"]
        κ = initialCondition["κ"]
        κc = initialCondition["κc"]    
    end

    pos = 0:steps:chains*steps

    #Sample
    for chainId in 1:chains
                
        #Make ignorable steps
        mcmcSampleGammaStep!( data, datac, k, w, wc, θ, θc, κ, κc, ignore)
        
        #Make steps in gap
        for i in 1:steps
            mcmcSampleGammaStep!( data, datac, k, w, wc, θ, θc, κ, κc, gap)
            #Save
            out["w"][pos[chainId]+i,:] .= w
            out["θ"][pos[chainId]+i,:] = θ
            out["κ"][pos[chainId]+i,:] = κ
            out["wc"][pos[chainId]+i,:] = wc
            out["θc"][pos[chainId]+i,:] = θc
            out["κc"][pos[chainId]+i,:] = κc           
        end 
    end
    
    return out

end


In [9]:
norm = Gamma(1.,2.)
norm2 = Gamma(2.,3.)
N = 10000

x = rand(norm,N)
xx = rand(norm2,N).+rand(norm,N)

init = Dict()
init["w"] = [.5,.5]
init["θ"] = [100.,1.]
init["κ"] = [7,2]
init["wc"] = [.5,.5]
init["θc"] = [1000.,2.]
init["κc"] = [8,2]

@btime n=mcmcSample(x,xx,[2 2],basis=:Gamma,initialCondition=init,steps=1000)

LoadError: UndefVarError: sliceSamplingLog not defined